In [7]:
START_DATE = '2000-09-01'
END_DATE = '2023-08-31'
CENTER = 'SAC'
All_DATES = f'https://api.avalanche.org/v2/public/products?avalanche_center_id={CENTER}&date_start={START_DATE}&date_end={END_DATE}'
dietailed_report_tmplt = 'https://api.avalanche.org/v2/public/product/{id_}'

In [39]:
import requests
from tqdm.notebook import tqdm
import pandas as pd
import functools

In [9]:
@functools.lru_cache
def get_all_reports(url):
    response = requests.get(url)

response = get_all_reports(All_DATES)

In [10]:
all_reports = response.json()
forcast_reports = [report for report in all_reports if report['product_type'] == 'forecast']

In [43]:
@functools.lru_cache
def get_report(report_url):
    res = requests.get(report_url)
    data = res.json()
    return data

In [55]:
def parse_report(report):
    id_ = report['id']
    date_str, time_str = report['start_date'].split('T')
    date = datetime.datetime.strptime(date_str, '%Y-%m-%d')
    rating = report['danger_level_text']
    report_url = dietailed_report_tmplt.format(id_=id_)
    data = get_report(report_url)
    problems = []
    for problem in data['forecast_avalanche_problems']:
        problems.append(problem['name'])
    return date, rating, report_url, tuple(problems)

In [56]:
data = []
for report in tqdm(forcast_reports):
    data.append(parse_report(report))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 521/521 [01:54<00:00,  4.57it/s]


In [62]:
df = pd.DataFrame(data, columns=['date', 'rating', 'report', 'problems'])

In [140]:
# df.head()
# _df = df.copy()
df = _df.copy()

In [153]:
dups = df.duplicated(['date'], keep=False)
dups.sum()

0

In [154]:
# df.to_pickle(f'../data/avy_data_new.pkl')